<a href="https://colab.research.google.com/github/Naoki0424/my_document/blob/main/tensorflow_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low-level APIを触ってみる

## TensorFlowのVerを下げる
Low-level APIを動かすためにはTensorFlowのVerを下げる必要があります

In [22]:
%tensorflow_version 1.x

## ライブラリをインポート

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## 事前データを作成

In [24]:
# 目的変数
train_t = np.array([5.2, 5.7, 6.7, 8.9, 13.2, 15.3, 20.5, 29.2, 20.0, 19.2, 10.2, 9])
train_t = train_t.reshape([12, 1])
# 説明変数
train_x = np.array([[mon**n for n in range(0, 5)] for mon in range(1, 13)])

In [25]:
train_t

array([[ 5.2],
       [ 5.7],
       [ 6.7],
       [ 8.9],
       [13.2],
       [15.3],
       [20.5],
       [29.2],
       [20. ],
       [19.2],
       [10.2],
       [ 9. ]])

In [26]:
train_x

array([[    1,     1,     1,     1,     1],
       [    1,     2,     4,     8,    16],
       [    1,     3,     9,    27,    81],
       [    1,     4,    16,    64,   256],
       [    1,     5,    25,   125,   625],
       [    1,     6,    36,   216,  1296],
       [    1,     7,    49,   343,  2401],
       [    1,     8,    64,   512,  4096],
       [    1,     9,    81,   729,  6561],
       [    1,    10,   100,  1000, 10000],
       [    1,    11,   121,  1331, 14641],
       [    1,    12,   144,  1728, 20736]])

## 予測値を計算する式を定義する
例えば式を y = Xwとする時に、これをTensorFlow(Low-level API)のコードとして実装する場合は準備が必要です。

入力データをTensorFlowに受け渡す際はTensorFlow側でデータを受け取るための箱を用意する必要があります。

これは`Placeholder`と呼ばれています。

In [27]:
x = tf.placeholder(tf.float32, [None, 5])

placeholderの一つ目の引数はデータ方の指定をしています。基本的にデータ方を設定するのはLow-level APIのみです。

２つ目は受け取るデータの形状を指定します。今回の場合はtrain_xを受け取るため形状は[12, 5]となるはずですが、12ではなくNoneが指定されています。これは任意の個数を受け取れるようにするためです。

機械学習の学習処理においては一度に幾つかのデータをまとめて受け渡しますが、具体的な数字は場合によって異なります。そのためPlaceholderを定義する際は最初の次元をNoneにすることで任意の個数を受け取れるようにしています

パラメータ調整の対象となる係数（上の式で言うw）にはVariableというオブジェクトを定義します。

Variableは学習により最適化が行われる係数です

In [29]:
w = tf.Variable(tf.zeros([5, 1]))

`tf.zeros[5, 1]`は初期値を表しています

$$
w =\begin{pmatrix}
    w_{0} \\
    \vdots \\
    w_{4}
\end{pmatrix}
=
\begin{pmatrix}
    0 \\
    \vdots \\
    0
\end{pmatrix} \quad
$$

これにより式の定義を行うことができます。TensorFlowでは行列の積を`matmul`で計算します。

In [30]:
y = tf.matmul(x, w)

予測値を計算する式の定義を行うことができました。

## パラメータの最適化で使用する誤差関数の定義を行う

定義する誤差関数

$$
E = \frac{1}{2}\sum_{n=1}^{12}(y_{n} - t_{n})^2
$$

上の式の変数は以下を表している

$$y_{n} = 予測値$$
$$t_{n} = 正解ラベル$$

正解ラベルにはtrain_tが代入されますが、train_xの時と同じくPlaceholderを定義する必要があります。

In [31]:
t = tf.placeholder(tf.float32, [None, 1])

これらを組み合わせることで誤差関数は以下のように定義できます

In [32]:
loss = tf.reduce_sum(tf.square(y - t))

(y-t)はベクトルの引き算。square()はベクトルの各成分を2乗する組み込み関数です。
reduce_sum()はベクトルの各成分を全て足し合わせるという組み込み関数です。先頭の1/2は誤差関数を最小化するwを求める上では不要なので省いています。

## 学習処理